In [1]:
import pickle
from glob import glob

files = sorted(glob('/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/*.pkl'))
files = files[len(files) // 2:]
len(files)

3812

In [2]:
from datasets import Audio
from tqdm import tqdm
import numpy as np
import IPython.display as ipd

In [3]:
sr = 16000
reader = Audio(sampling_rate = sr)

In [4]:
with open(files[0], 'rb') as fopen:
    data = pickle.load(fopen)

In [5]:
max(data[0]['asr_model'][0], key=data[0]['asr_model'][0].get)

'ms'

In [6]:
data[0]

{'wav_data': '/home/husein/ssd2/processed-youtube-asr-whisper-large-v3/Belajarlah_Memaafkan_Orang_Lain_-_Ustaz_Adnin_Roslan_[kXJUgpeeQWU]/0.mp3',
 'start': 0.0,
 'asr_model': ({'haw': 0.0004717402916867286,
   'tr': 0.0009716476779431105,
   'fa': 0.00021259349887259305,
   'hi': 0.00020032981410622597,
   'vi': 0.0005049401079304516,
   'ta': 4.902168075204827e-05,
   'et': 4.332884600444231e-06,
   'mg': 3.358373987794039e-07,
   'am': 1.839123069657944e-05,
   'gu': 1.6431578842457384e-05,
   'be': 4.886377951152099e-07,
   'nn': 0.007512533105909824,
   'lv': 1.2892178347101435e-05,
   'ko': 0.00030599598539993167,
   'sn': 0.0001415890292264521,
   'hr': 7.0581018007942475e-06,
   'pl': 3.6917819670634344e-05,
   'bs': 4.124826318729902e-06,
   'is': 1.9146273189107887e-06,
   'bn': 5.781698928331025e-05,
   'ja': 0.00048791812150739133,
   'ln': 0.00030045805033296347,
   'gl': 1.590410761309613e-06,
   'ms': 0.945112407207489,
   'af': 3.4661305107874796e-05,
   'ar': 0.00028200

In [7]:
ms_score = [s['avg_logprob'] for s in data[0]['asr_model'][1]['segments']]
en_score = [s['avg_logprob'] for s in data[0]['asr_model'][2]['segments']]

if np.mean(ms_score) > np.mean(en_score):
    selected = data[0]['asr_model'][1]['segments']
else:
    selected = data[0]['asr_model'][2]['segments']

In [8]:
i = 0

s = selected[i]['start']
e = selected[i]['end']
t = selected[i]['text']
s, e, t

(0.0, 1.7, ' bismillahirrahmanirrahim, insyaAllah')

In [9]:
f = data[0]['wav_data']
audio = reader.decode_example(reader.encode_example(f))['array']
y = audio[int(s * sr): int(e * sr)]

In [10]:
ipd.Audio(y, rate = sr)

In [11]:
import malaya_speech

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [12]:
vad = malaya_speech.vad.webrtc()

In [13]:
y_int = malaya_speech.astype.float_to_int(y)
frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr))
np.mean([vad(frame) for frame in frames_int])

0.9473684210526315

In [14]:
import malaya
from malaya.text.normalization import cardinal

tokenizer = malaya.tokenizer.Tokenizer(hypen = False, parliament = False, time = False, time_pukul = False,
                                      temperature = False, distance = False, volume = False, duration = False,
                                      weight = False, date = False, money = False)

In [15]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    tokenized = tokenizer.tokenize(string)
    string = ' '.join(tokenized)
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [16]:
directory = '/home/husein/ssd3/stt/whisper-v4'

In [17]:
!mkdir {directory}

mkdir: cannot create directory ‘/home/husein/ssd3/stt/whisper-v4’: File exists


In [18]:
import torchaudio
import torch
import os

In [19]:
dataset = []
index = 0
for no, f in enumerate(files):
    
    with open(f, 'rb') as fopen:
        data = pickle.load(fopen)
    
    for i in tqdm(range(len(data))):
        lang = max(data[i]['asr_model'][0], key=data[i]['asr_model'][0].get)
        if lang not in {'ms', 'en'}:
            continue
        
        ms_score = [s['avg_logprob'] for s in data[i]['asr_model'][1]['segments']]
        en_score = [s['avg_logprob'] for s in data[i]['asr_model'][2]['segments']]

        if np.mean(ms_score) > np.mean(en_score):
            selected = data[i]['asr_model'][1]['segments']
        else:
            selected = data[i]['asr_model'][2]['segments']
            
        audio = reader.decode_example(reader.encode_example(data[i]['wav_data']))['array']
        for k in range(len(selected)):
            s = selected[k]['start']
            e = selected[k]['end'] + 0.1
            t = selected[k]['text']
            cleaned_t = preprocessing_text(t)
            if not len(cleaned_t):
                continue
                
            audio_path = os.path.join(directory, f'{index}.mp3')
            a = audio[int(s * sr): int(e * sr)]
            torchaudio.save(audio_path, 
                            torch.tensor(a.astype('float32')).unsqueeze(0), 
                            16000, format='mp3')
            try:
                y_int = malaya_speech.astype.float_to_int(a)
                frames_int = list(malaya_speech.utils.generator.frames(y_int, 30, sr, append_ending_trail = False))

                dataset.append(
                    {
                        'file': audio_path,
                        'text': t,
                        'cleaned': cleaned_t,
                        'no': no,
                        'i': i,
                        'k': k,
                        'avg_logprob': selected[k]['avg_logprob'],
                        'no_speech_prob': selected[k]['no_speech_prob'],
                        'vad': float(np.mean([vad(frame) for frame in frames_int])),
                    }
                )
            except:
                pass
            index += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:06<00:00,  3.92it/s]


 76%|██████████████████████████████████████████████████████████████████████▊                      | 48/63 [00:08<00:03,  4.51it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:01<00:00,  3.27it/s]


 74%|████████████████████████████████████████████████████████████████████▉                        | 43/58 [00:11<00:03,  3.95it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 12%|██████████▌                                                                                 | 99/859 [00:27<03:34,  3.54it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████████████████████████████████████████████

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:01<00:00,  4.04it/s]


 69%|███████████████████████████████████████████████████████████████▉                             | 11/16 [00:03<00:01,  3.08it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:02<00:00,  3.56it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:03<00:00,  4.83it/s]


 33%|██████████████████████████████▏                                                             | 40/122 [00:08<00:20,  4.05it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:15<00:00,  5.37it/s]


 71%|██████████████████████████████████████████████████████████████████                           | 22/31 [00:04<00:01,  4.74it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:09<00:00,  4.58it/s]


 47%|███████████████████████████████████████████▍                                                 | 14/30 [00:03<00:03,  4.82it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|███████████████████████████████████████████████████████████████████████████████████████████| 449/449 [01:19<00:00,  5.68it/s]


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 84/84 [00:12<00:00,  6.80it/s]


In [20]:
len(dataset)

1527921

In [21]:
import json

with open('dataset-whisper-stt-v4.json', 'w') as fopen:
    json.dump(dataset, fopen)

In [23]:
!ls -lh dataset-whisper-stt-v*.json

-rw-r--r-- 1 husein husein 523M Jun  15 15:39 dataset-whisper-stt-v2.json
-rw-r--r-- 1 husein husein 308M Jun  15 15:39 dataset-whisper-stt-v3.json
-rw-r--r-- 1 husein husein 435M Jun  15 15:39 dataset-whisper-stt-v4.json
